In [2]:
import pandas as pd 
import numpy as np
from collections import defaultdict

In [3]:
df = pd.read_csv(r"C:\Users\araca\Documents\GitHub\c2425-R4\src\bases de datos\primeras_peleas_ufc.csv")
df


,Peleador_A,Peleador_B,DATE,WINNER,METHOD,TIME,ROUND,KD_A,KD_B,SIG_STR_A,...,STR_BODY_A,STR_BODY_B,STR_LEG_A,STR_LEG_B,STR_DISTANCE_A,STR_DISTANCE_B,STR_CLINCH_A,STR_CLINCH_B,STR_GROUND_A,STR_GROUND_B
0,Dricus Du Plessis,Sean Strickland,"February 15, 2025",0,Decision - Unanimous,TIME: 5:00,ROUND: 5,0,0,46%,...,45 of 59,29 of 41,52 of 62,9 of 10,147 of 314,128 of 263,0 of 0,0 of 0,0 of 0,0 of 0
1,Zhang Weili,Tatiana Suarez,"February 15, 2025",0,Decision - Unanimous,TIME: 5:00,ROUND: 5,0,0,60%,...,16 of 21,2 of 6,5 of 10,9 of 14,29 of 62,10 of 33,11 of 11,2 of 3,13 of 14,0 of 0
2,Justin Tafa,Tallison Teixeira,"February 15, 2025",1,KO/TKO,TIME: 0:35,ROUND: 1,0,1,0%,...,0 of 0,3 of 3,0 of 0,0 of 0,0 of 1,1 of 2,0 of 0,3 of 3,0 of 0,4 of 4
3,Jimmy Crute,Rodolfo Bellato,"February 15, 2025",1,Decision - Majority,TIME: 5:00,ROUND: 3,1,0,50%,...,8 of 10,24 of 29,5 of 9,29 of 36,77 of 170,85 of 136,11 of 13,4 of 5,18 of 26,0 of 0
4,Jake Matthews,Francisco Prado,"February 15, 2025",0,Decision - Unanimous,TIME: 5:00,ROUND: 3,0,0,36%,...,24 of 33,19 of 28,5 of 5,23 of 26,56 of 169,47 of 132,12 of 15,11 of 13,0 of 0,1 of 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114,Deron Winn,Antonio Arroyo,"December 19, 2020",0,Decision - Unanimous,TIME: 5:00,ROUND: 3,0,0,52%,...,5 of 7,11 of 12,5 of 5,5 of 5,18 of 35,43 of 60,2 of 2,0 of 0,0 of 1,2 of 2
2115,Gillian Robertson,Taila Santos,"December 19, 2020",1,Decision - Unanimous,TIME: 5:00,ROUND: 3,0,0,64%,...,1 of 1,3 of 4,1 of 1,3 of 3,4 of 9,4 of 13,0 of 0,1 of 1,5 of 5,22 of 23
2116,Tafon Nchukwi,Jamie Pickett,"December 19, 2020",0,Decision - Unanimous,TIME: 5:00,ROUND: 3,1,0,64%,...,49 of 53,8 of 9,1 of 1,4 of 4,59 of 118,39 of 77,57 of 63,1 of 4,4 of 6,0 of 0
2117,Jimmy Flick,Cody Durden,"December 19, 2020",0,Submission,TIME: 3:18,ROUND: 1,0,0,42%,...,0 of 1,3 of 3,2 of 6,1 of 1,6 of 14,17 of 28,0 of 0,2 of 2,0 of 0,2 of 2


In [4]:
df["METHOD"].unique()

array(['Decision - Unanimous', 'KO/TKO', 'Decision - Majority',
       'Submission', 'Decision - Split', "TKO - Doctor's Stoppage",
       'Could Not Continue', 'DQ', 'Overturned'], dtype=object)

In [17]:
# Inicializar diccionario de puntuaciones
ranking = defaultdict(lambda: 100)  # Empezamos con 100 puntos por peleador

# Definir pesos para los métodos de victoria
pesos = {"KO/TKO" or "TKO": 1, "Submission": 0.5, "Decision - Unanimous" or "Decision - Majority": 0.25}

def actualizar_puntuacion(ganador, perdedor, metodo):
    peso = pesos.get(metodo, 0.2)  # Por defecto, 0.5 si el método no está especificado
    factor = ranking[perdedor] / 100  # Se pondera según el nivel del rival
    ranking[ganador] += peso * factor * 10  # Se ajusta la puntuación
    ranking[perdedor] -= (1 - peso) * factor * 5  # Penalización por perder

# Iterar sobre cada pelea y actualizar el ranking
for _, row in df.iterrows():
    if(row['WINNER'] == 0):
        actualizar_puntuacion(row['Peleador_A'], row['Peleador_B'], row['METHOD'])
    else:
        actualizar_puntuacion(row['Peleador_B'], row['Peleador_A'], row['METHOD'])
# Convertir el ranking a un DataFrame y ordenarlo
df_ranking = pd.DataFrame(sorted(ranking.items(), key=lambda x: x[1], reverse=True), columns=['Peleador', 'Puntuación'])

# Mostrar el top 10
df_ranking


,Peleador,Puntuación
0,Alex Pereira,178.689343
1,Steve Garcia,161.990875
2,Kennedy Nzechukwu,160.127987
3,Islam Makhachev,157.535549
4,Tom Aspinall,154.600678
...,...,...
1010,Jimmy Crute,86.261760
1011,Tony Ferguson,85.864832
1012,Dan Argueta,85.589254
1013,Ji Yeon Kim,85.377600
